In [1]:
import pandas as pd

df = pd.read_excel('최종.xlsx')  # CSV 파일로 변경

In [3]:
df['기준년월']

0      2025-04-01
1      2025-04-01
2      2025-04-01
3      2025-04-01
4      2025-04-01
          ...    
2209   2022-12-01
2210   2022-12-01
2211   2022-12-01
2212   2022-12-01
2213   2022-12-01
Name: 기준년월, Length: 2214, dtype: datetime64[ns]

In [13]:
import pandas as pd

# 1. 파일 이름 정의
main_data_file = '최종.xlsx'
kb_data_file = '월간 아파트 매매가격지수_20250723.xlsx' 

try:
    # 2. 각 파일 불러오기
    df_main = pd.read_excel(main_data_file)
    # KB 데이터는 헤더가 2번째 줄에 있으므로 skiprows=1 로 설정
    try:
        df_kb_raw = pd.read_excel(kb_data_file)
    except UnicodeDecodeError:
        df_kb_raw = pd.read_excel(kb_data_file, skiprows=1, encoding='cp949')

    print("✅ 두 파일 로드 성공!")

    # 3. KB 매매가격지수 데이터 전처리
    # 첫 번째 열의 이름을 '지역명'으로 변경
    df_kb_raw.rename(columns={df_kb_raw.columns[0]: '지역명'}, inplace=True)
    
    # '지역명'을 인덱스로 설정
    df_kb_raw.set_index('지역명', inplace=True)

    # '전국' 행의 데이터만 선택하여 Wide -> Long 형태로 변환
    series_kb_nationwide = df_kb_raw.loc['전국']
    df_kb_long = series_kb_nationwide.to_frame(name='KB매매가격지수')
    
    # 인덱스(날짜)를 컬럼으로 이동하고, 날짜 형식으로 변환
    df_kb_long.reset_index(inplace=True)
    df_kb_long.rename(columns={'index': '날짜'}, inplace=True)
    # 월간 데이터이므로 날짜 변환이 더 간단해집니다.
    df_kb_long['날짜'] = pd.to_datetime(df_kb_long['날짜'], errors='coerce')
    
    # 결합을 위한 'merge_key' 생성 (YYYY-MM 형식)
    df_kb_long['merge_key'] = df_kb_long['날짜'].dt.strftime('%Y-%m')
    
    print("✅ KB 월간 데이터 처리 완료.")

    # 4. 기존 데이터 전처리
    df_main['기준년월'] = pd.to_datetime(df_main['기준년월'])
    df_main['merge_key'] = df_main['기준년월'].dt.strftime('%Y-%m')

    # 5. 두 데이터 결합 (Merge)
    df_merged = pd.merge(df_main, df_kb_long[['merge_key', 'KB매매가격지수']], on='merge_key', how='left')

    # 사용이 끝난 'merge_key' 컬럼 제거
    df_merged = df_merged.drop(columns=['merge_key'])
    
    # KB매매가격지수가 없는 행(과거 데이터 등)은 이전 값으로 채우기 (forward-fill)
    df_merged['KB매매가격지수'].fillna(method='ffill', inplace=True)

    # 6. 최종 결과 저장 및 확인
    output_filename = 'final_merged_data.csv'
    df_merged.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    print("\n🎉 데이터 결합 완료!")
    print(f"최종 결과가 '{output_filename}' 파일로 저장되었습니다.")
    print("\n--- 결합된 데이터 상위 5개 행 ---")
    print(df_merged[['기준년월', '아파트', '분양률', 'KB매매가격지수']].head())

except FileNotFoundError as e:
    print(f"오류: 파일을 찾을 수 없습니다. 파일 이름이 정확한지, 두 파일이 모두 업로드 되었는지 확인해주세요.")
    print(f"에러 메시지: {e}")
except KeyError as e:
    print(f"오류: 데이터에서 '{e}' 키(열 또는 행 이름)를 찾을 수 없습니다. KB부동산에서 받은 파일의 형식이 예상과 다른 것 같습니다.")
except Exception as e:
    print(f"알 수 없는 오류가 발생했습니다: {e}")

✅ 두 파일 로드 성공!
✅ KB 월간 데이터 처리 완료.

🎉 데이터 결합 완료!
최종 결과가 'final_merged_data.csv' 파일로 저장되었습니다.

--- 결합된 데이터 상위 5개 행 ---
        기준년월        아파트       분양률   KB매매가격지수
0 2025-04-01  강릉자이르네디오션  0.962264  89.303238
1 2025-04-01  강릉자이르네디오션  1.000000  89.303238
2 2025-04-01  강릉자이르네디오션  0.961538  89.303238
3 2025-04-01  강릉자이르네디오션  0.925000  89.303238
4 2025-04-01  강릉자이르네디오션  1.000000  89.303238


/var/folders/kq/wzvd01qd1ql50sz17pwmt9b00000gn/T/ipykernel_33521/2168569126.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['KB매매가격지수'].fillna(method='ffill', inplace=True)
/var/folders/kq/wzvd01qd1ql50sz17pwmt9b00000gn/T/ipykernel_33521/2168569126.py:51: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged['KB매매가격지수'].fillna(method='ffill', inplace=True)
/var/folders/kq/wzvd01qd1ql50sz17pwmt9b00000gn/T/ipykernel_33521/21

In [14]:
df_merged

,기준년월,지역,아파트,공급면적(㎡),전용면적(㎡),일반분양,특별분양,세대수,미분양수,분양가(만원),...,중학교(2km 이내),고등학교(2km 이내),지하철 - 반경 1.5km 이내,버스 - 반경 500m 이내,고속철도 - 10km 이내,고속도로IC - 10km 이내,주변시세 평균,주변시세 평균(만원),시세차익(만원),KB매매가격지수
0,2025-04-01,강원도,강릉자이르네디오션,109.5400,84.8800,26,27,53,2,57590,...,1,1,0,0,1,3,3.9,39000,-18590,89.303238
1,2025-04-01,강원도,강릉자이르네디오션,109.9600,84.9800,10,11,21,0,50260,...,1,1,0,0,1,3,3.9,39000,-11260,89.303238
2,2025-04-01,강원도,강릉자이르네디오션,109.9700,84.9800,11,15,26,1,56360,...,1,1,0,0,1,3,3.9,39000,-17360,89.303238
3,2025-04-01,강원도,강릉자이르네디오션,111.1500,84.9200,19,21,40,3,53700,...,1,1,0,0,1,3,3.9,39000,-14700,89.303238
4,2025-04-01,강원도,강릉자이르네디오션,131.9600,101.9600,2,0,2,0,72320,...,1,1,0,0,1,3,3.9,39000,-33320,89.303238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,2022-12-01,충청북도,음성동문디이스트,110.3724,84.9615,189,498,687,34,32600,...,0,1,0,4,0,3,1.1,11000,-21600,96.569873
2210,2022-12-01,충청북도,진천금호어울림센트럴파크,99.8234,76.9848,38,38,76,23,25940,...,1,0,0,5,0,3,1.9,19000,-6940,96.569873
2211,2022-12-01,충청북도,진천금호어울림센트럴파크,99.8946,76.9209,22,17,39,0,25440,...,1,0,0,5,0,3,1.9,19000,-6440,96.569873
2212,2022-12-01,충청북도,진천금호어울림센트럴파크,109.6959,84.9840,71,75,146,110,28510,...,1,0,0,5,0,3,1.9,19000,-9510,96.569873


In [56]:
import yfinance as yf
import pandas as pd

# 1. KOSPI 데이터 불러오기
print("야후 파이낸스에서 KOSPI 데이터를 불러오는 중...")

kospi_daily = yf.download('^KS11', start='2021-12-01', end='2025-04-01', interval='1d')
print("데이터 로드 완료!")

# 2. 일별 데이터를 월별 평균 데이터로 가공
print("\n일별 데이터를 월별 평균 데이터로 변환 중...")

# 월별 평균 계산 (결과가 데이터프레임으로 반환됨)
kospi_monthly_df = kospi_daily['Close'].resample('ME').mean()

# =================================================================
# ## (오류 수정) .to_frame() 대신 .rename()으로 컬럼명 변경 ##
# =================================================================
# 'Close' 컬럼의 이름을 'KOSPI_지수'로 올바르게 변경합니다.
df_kospi = kospi_monthly_df.rename(columns={'Close': 'KOSPI_지수'})
# =================================================================

# 3. 최종 결합 준비가 완료된 데이터 확인
# merge_key 생성을 위해 인덱스 리셋
df_kospi.reset_index(inplace=True)
df_kospi.rename(columns={'Date': '날짜'}, inplace=True)
df_kospi['merge_key'] = df_kospi['날짜'].dt.strftime('%Y-%m')

print("\n이제 이 데이터를 기존 메인 데이터와 결합(merge)하시면 됩니다.")

/var/folders/kq/wzvd01qd1ql50sz17pwmt9b00000gn/T/ipykernel_33521/3666318663.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  kospi_daily = yf.download('^KS11', start='2021-12-01', end='2025-04-01', interval='1d')
[*********************100%***********************]  1 of 1 completed

야후 파이낸스에서 KOSPI 데이터를 불러오는 중...
데이터 로드 완료!

일별 데이터를 월별 평균 데이터로 변환 중...

이제 이 데이터를 기존 메인 데이터와 결합(merge)하시면 됩니다.


In [29]:
kospi_daily = yf.download('^KS11', start='2022-12-01', end='2025-04-01', interval='1d')
print("데이터 로드 완료!")

# 2. 일별 데이터를 월별 평균 데이터로 가공
print("\n일별 데이터를 월별 평균 데이터로 변환 중...")

# 월별 평균 계산 (결과가 데이터프레임으로 반환됨)
kospi_monthly_df = kospi_daily['Close'].resample('ME').mean()

/var/folders/kq/wzvd01qd1ql50sz17pwmt9b00000gn/T/ipykernel_33521/4257604379.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  kospi_daily = yf.download('^KS11', start='2022-12-01', end='2025-04-01', interval='1d')
[*********************100%***********************]  1 of 1 completed

데이터 로드 완료!

일별 데이터를 월별 평균 데이터로 변환 중...


In [57]:
if 'Ticker' in kospi_monthly_df.columns:
    kospi_monthly_df.drop(columns=['Ticker'], inplace=True)
    print("✅ Ticker 컬럼 제거 완료.")  

In [58]:
from operator import index
import pandas as pd

# 원본 데이터프레임 예시
df = kospi_monthly_df

# 컬럼명과 인덱스 정리
df = df.rename(columns={'^KS11': 'Close'})
df = df.reset_index().rename(columns={'index': 'Date'})


In [59]:
# 인덱스를 초기화하고 Ticker 컬럼 제거
df = df.reset_index(drop=True)

In [60]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2214 entries, 0 to 2213
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   기준년월               2214 non-null   datetime64[ns]
 1   지역                 2214 non-null   object        
 2   아파트                2214 non-null   object        
 3   공급면적(㎡)            2214 non-null   float64       
 4   전용면적(㎡)            2214 non-null   float64       
 5   일반분양               2214 non-null   int64         
 6   특별분양               2214 non-null   int64         
 7   세대수                2214 non-null   int64         
 8   미분양수               2214 non-null   int64         
 9   분양가(만원)            2214 non-null   int64         
 10  분양률                2214 non-null   float64       
 11  준공여부               2214 non-null   object        
 12  건설사                2214 non-null   object        
 13  금리                 2214 non-null   float64       
 14  환율      

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    40 non-null     datetime64[ns]
 1   Close   40 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 772.0 bytes


In [62]:
df

Ticker,Date,Close
0,2021-12-31,2988.494074
1,2022-01-31,2859.066830
2,2022-02-28,2724.015015
3,2022-03-31,2698.716192
4,2022-04-30,2703.242850
5,2022-05-31,2629.215002
6,2022-06-30,2474.948486
7,2022-07-31,2362.794771
8,2022-08-31,2479.571367
9,2022-09-30,2333.928503


In [63]:
import pandas as pd

# 1. 공통 키인 'YearMonth'를 생성 (형식: '2023-01')
df['YearMonth'] = df['Date'].dt.to_period('M').astype(str)
df_merged['YearMonth'] = df_merged['기준년월'].dt.to_period('M').astype(str)

# 2. 'YearMonth' 기준으로 병합
df_result = pd.merge(df_merged, df[['YearMonth', 'Close']], on='YearMonth', how='left')

# 3. 병합 후 'YearMonth' 컬럼 제거 (선택)
df_result.drop(columns=['YearMonth'], inplace=True)


In [64]:
df_result

,기준년월,지역,아파트,공급면적(㎡),전용면적(㎡),일반분양,특별분양,세대수,미분양수,분양가(만원),...,고등학교(2km 이내),지하철 - 반경 1.5km 이내,버스 - 반경 500m 이내,고속철도 - 10km 이내,고속도로IC - 10km 이내,주변시세 평균,주변시세 평균(만원),시세차익(만원),KB매매가격지수,Close
0,2025-04-01,강원도,강릉자이르네디오션,109.5400,84.8800,26,27,53,2,57590,...,1,0,0,1,3,3.9,39000,-18590,89.303238,NaN
1,2025-04-01,강원도,강릉자이르네디오션,109.9600,84.9800,10,11,21,0,50260,...,1,0,0,1,3,3.9,39000,-11260,89.303238,NaN
2,2025-04-01,강원도,강릉자이르네디오션,109.9700,84.9800,11,15,26,1,56360,...,1,0,0,1,3,3.9,39000,-17360,89.303238,NaN
3,2025-04-01,강원도,강릉자이르네디오션,111.1500,84.9200,19,21,40,3,53700,...,1,0,0,1,3,3.9,39000,-14700,89.303238,NaN
4,2025-04-01,강원도,강릉자이르네디오션,131.9600,101.9600,2,0,2,0,72320,...,1,0,0,1,3,3.9,39000,-33320,89.303238,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,2022-12-01,충청북도,음성동문디이스트,110.3724,84.9615,189,498,687,34,32600,...,1,0,4,0,3,1.1,11000,-21600,96.569873,2361.278564
2210,2022-12-01,충청북도,진천금호어울림센트럴파크,99.8234,76.9848,38,38,76,23,25940,...,0,0,5,0,3,1.9,19000,-6940,96.569873,2361.278564
2211,2022-12-01,충청북도,진천금호어울림센트럴파크,99.8946,76.9209,22,17,39,0,25440,...,0,0,5,0,3,1.9,19000,-6440,96.569873,2361.278564
2212,2022-12-01,충청북도,진천금호어울림센트럴파크,109.6959,84.9840,71,75,146,110,28510,...,0,0,5,0,3,1.9,19000,-9510,96.569873,2361.278564


In [65]:
# '2025-04'에 해당하는 행의 Close 값이 NaN이면 2697.67로 채우기
df_result.loc[
    (df_result['기준년월'].dt.to_period('M') == '2025-04') & (df_result['Close'].isna()),
    'Close'
] = 2697.67


In [66]:
df_result.to_csv('final_merged_data.csv', index=False, encoding='utf-8-sig')

In [69]:
df_age = pd.read_csv('행정구역_시군구_별__성별_인구수_20250723172720_분석(전월_대비_증감,증감률).csv')

In [70]:
df_age

,행정구역(시군구)별,2021.01,2021.01.1,2021.01.2,2021.01.3,2021.01.4,2021.01.5,2021.01.6,2021.01.7,2021.01.8,...,2025.05.8,2025.06,2025.06.1,2025.06.2,2025.06.3,2025.06.4,2025.06.5,2025.06.6,2025.06.7,2025.06.8
0,행정구역(시군구)별,총인구수 (명),총인구수 (명),총인구수 (명),남자인구수 (명),남자인구수 (명),남자인구수 (명),여자인구수 (명),여자인구수 (명),여자인구수 (명),...,여자인구수 (명),총인구수 (명),총인구수 (명),총인구수 (명),남자인구수 (명),남자인구수 (명),남자인구수 (명),여자인구수 (명),여자인구수 (명),여자인구수 (명)
1,행정구역(시군구)별,원데이터,전월 대비 증감,증감률,원데이터,전월 대비 증감,증감률,원데이터,전월 대비 증감,증감률,...,증감률,원데이터,전월 대비 증감,증감률,원데이터,전월 대비 증감,증감률,원데이터,전월 대비 증감,증감률
2,전국,51825932,-3091,0.0,25838512,-2517,0.0,25987420,-574,0.0,...,0.0,51164582,-4566,0.0,25467115,-3283,0.0,25697467,-1283,0.0
3,서울특별시,9657969,-10496,-0.1,4695999,-5724,-0.1,4961970,-4772,-0.1,...,0.0,9325616,-2426,0.0,4497354,-1791,0.0,4828262,-635,0.0
4,부산광역시,3389388,-2558,-0.1,1659527,-1473,-0.1,1729861,-1085,-0.1,...,-0.1,3252830,-1627,0.0,1582043,-885,-0.1,1670787,-742,0.0
5,대구광역시,2415813,-2533,-0.1,1191656,-1453,-0.1,1224157,-1080,-0.1,...,0.0,2357052,-945,0.0,1155353,-581,-0.1,1201699,-364,0.0
6,인천광역시,2942452,-376,0.0,1473639,-304,0.0,1468813,-72,0.0,...,0.1,3039450,2401,0.1,1518758,1260,0.1,1520692,1141,0.1
7,광주광역시,1449115,-947,-0.1,716929,-419,-0.1,732186,-528,-0.1,...,-0.1,1399082,-798,-0.1,690429,-397,-0.1,708653,-401,-0.1
8,대전광역시,1463291,-591,0.0,730396,-303,0.0,732895,-288,0.0,...,0.0,1439764,157,0.0,717456,-55,0.0,722308,212,0.0
9,울산광역시,1135370,-647,-0.1,583402,-306,-0.1,551968,-341,-0.1,...,-0.1,1093665,-362,0.0,563241,-98,0.0,530424,-264,0.0


In [82]:
import pandas as pd

# 1. 파일을 처음부터 읽되, header=None으로 읽어서 수동으로 처리
df_raw = pd.read_csv("인구.csv", header=None)

# 2. 진짜 컬럼명은 두 번째 행 (index=1) → 이걸 컬럼으로 쓰자
new_columns = df_raw.iloc[1].tolist()
df = df_raw[2:].copy()
df.columns = new_columns

# 3. 첫 번째 컬럼 이름 변경
df.rename(columns={df.columns[0]: "지역"}, inplace=True)

# 4. 필요한 행 필터링 (예: '전국계' 같은 합계 제거)
df = df[~df["지역"].str.contains("계|총", na=False)]

# 5. melt로 wide → long 변환
df_long = df.melt(id_vars="지역", var_name="기준년월", value_name="총인구수")

# 6. 타입 변환
df_long["기준년월"] = pd.to_datetime(df_long["기준년월"], format="%Y.%m", errors="coerce")
df_long["총인구수"] = pd.to_numeric(df_long["총인구수"], errors="coerce")

# 7. 정렬
df_long = df_long.sort_values(["지역", "기준년월"]).reset_index(drop=True)

# 8. 증감률 계산
df_long["증감률"] = df_long.groupby("지역")["총인구수"].pct_change() * 100


/var/folders/kq/wzvd01qd1ql50sz17pwmt9b00000gn/T/ipykernel_33521/767787416.py:28: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_long["증감률"] = df_long.groupby("지역")["총인구수"].pct_change() * 100


In [83]:
df_long

,지역,기준년월,총인구수,증감률
0,강원특별자치도,NaT,1541696.0,NaN
1,강원특별자치도,NaT,-1144.0,-1.000742e+02
2,강원특별자치도,NaT,-0.1,-9.999126e+01
3,강원특별자치도,NaT,775727.0,-7.757271e+08
4,강원특별자치도,NaT,-778.0,-1.001003e+02
...,...,...,...,...
9229,행정구역(시군구)별,NaT,NaN,NaN
9230,행정구역(시군구)별,NaT,NaN,NaN
9231,행정구역(시군구)별,NaT,NaN,NaN
9232,행정구역(시군구)별,NaT,NaN,NaN


In [89]:
import pandas as pd

def preprocess_population_data(file_path):
    """
    3중 헤더를 가진 인구 데이터를 전처리하는 함수입니다.
    '총인구수'와 '인구증감률'만 추출하여 데이터를 재구조화합니다.
    """
    df = pd.read_csv(file_path, header=[0, 1, 2], encoding='utf-8')

    region_col_name = df.columns[0]
    region_df = df[[region_col_name]]
    region_df.columns = ['지역']

    data_df = df.iloc[:, 1:]

    df_total_pop = data_df.xs('총인구수 (명)', level=1, axis=1)

    df_raw_data = df_total_pop.xs('원데이터', level=1, axis=1)
    df_rate_data = df_total_pop.xs('증감률', level=1, axis=1)

    df_raw = pd.concat([region_df, df_raw_data], axis=1)
    df_rate = pd.concat([region_df, df_rate_data], axis=1)

    pop_melted = df_raw.melt(id_vars='지역', var_name='기준년월', value_name='총인구수')
    rate_melted = df_rate.melt(id_vars='지역', var_name='기준년월', value_name='인구증감률')

    processed_df = pd.merge(pop_melted, rate_melted, on=['지역', '기준년월'])

    processed_df['기준년월'] = pd.to_datetime(processed_df['기준년월'], format='%Y.%m').dt.strftime('%Y-%m')
    processed_df['지역'] = processed_df['지역'].apply(lambda x: str(x).split('(')[0].strip())
    
    return processed_df

def merge_data(final_data_path, pop_data_df):
    """
    기존 데이터와 인구 데이터를 병합하는 함수입니다.
    """
    df_final = pd.read_csv(final_data_path)

    df_final['기준년월'] = pd.to_datetime(df_final['기준년월']).dt.strftime('%Y-%m')

    # '지역' 컬럼의 값 형식을 통일합니다.
    # '인구.csv'의 '특별자치도' 등을 'final_merged_data.csv'의 '도' 형식으로 맞춰줍니다.
    # 예: "강원특별자치도" -> "강원도"
    name_mapping = {
        '강원특별자치도': '강원도',
        '전북특별자치도': '전라북도', # '전북특별자치도'는 '전라북도'로 변경
        '제주특별자치도': '제주도'
    }
    pop_data_df['지역'] = pop_data_df['지역'].replace(name_mapping)

    # '기준년월'과 '지역'을 기준으로 두 데이터를 병합합니다.
    merged_df = pd.merge(df_final, pop_data_df, on=['기준년월', '지역'], how='left')

    return merged_df

# --- 스크립트 실행 부분 ---

# 파일 경로 설정
population_file = '인구.csv'
final_data_file = 'final_merged_data.csv'
output_file = 'merged_population_data.csv'

# 1. 인구 데이터 전처리
processed_pop_df = preprocess_population_data(population_file)

# 2. 전처리된 인구 데이터와 기존 데이터 병합
final_merged_df = merge_data(final_data_file, processed_pop_df)

# 결과 확인
print("--- 병합 후 데이터 샘플 (상위 5개) ---")
print(final_merged_df.head())

print("\n--- NaN이 아닌 데이터가 있는지 확인 (총인구수 기준) ---")
print(final_merged_df.dropna(subset=['총인구수']).head())


# 결과를 CSV 파일로 저장
final_merged_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"\n✅ 성공적으로 데이터를 병합하여 '{output_file}' 파일로 저장했습니다.")

--- 병합 후 데이터 샘플 (상위 5개) ---
      기준년월   지역        아파트  공급면적(㎡)  전용면적(㎡)  일반분양  특별분양  세대수  미분양수  분양가(만원)  \
0  2025-04  강원도  강릉자이르네디오션   109.54    84.88    26    27   53     2    57590   
1  2025-04  강원도  강릉자이르네디오션   109.96    84.98    10    11   21     0    50260   
2  2025-04  강원도  강릉자이르네디오션   109.97    84.98    11    15   26     1    56360   
3  2025-04  강원도  강릉자이르네디오션   111.15    84.92    19    21   40     3    53700   
4  2025-04  강원도  강릉자이르네디오션   131.96   101.96     2     0    2     0    72320   

   ...  버스 - 반경 500m 이내 고속철도 - 10km 이내 고속도로IC - 10km 이내  주변시세 평균  주변시세 평균(만원)  \
0  ...                0              1                3      3.9        39000   
1  ...                0              1                3      3.9        39000   
2  ...                0              1                3      3.9        39000   
3  ...                0              1                3      3.9        39000   
4  ...                0              1                3      3.9        39000   

   시